<a href="https://colab.research.google.com/github/cagBRT/Machine-Learning/blob/master/PCA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we look at high dimensional data: hand drawn digits.<br>
The digits are images 8x8 pixels.<br>
This means a supervised classification model will have 64 inputs.<br>
Can we reduce the number of inputs to the model?

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/Machine-Learning.git cloned-repo
%cd cloned-repo
print("done")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

Load the data: hand drawn digits<br>

The data consists of 8×8 pixel images, meaning that they are 64-dimensional (or features)<br>

We can use PCA to project them to a more manageable number of dimensions, say two:

In [ ]:
digits = load_digits()
digits.data.shape

In [ ]:
X = digits.images.reshape((len(digits.images), -1))
y = digits.target

**Create a model to classifiy the digits** 

In [ ]:
model_orig = DecisionTreeClassifier()
model_orig.fit(X, y)
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model_orig, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

**For the ASSIGNMENT**
Change the number of dimensions here

In [ ]:
pca = PCA(2)  # project from 64 to 2 dimensions
#fit the model to the data
projected = pca.fit_transform(X)
print(X.shape)
print(projected.shape)


We can now plot the first two principal components of each point to learn about the data. <br>
There are 10 clusters, one for each digit between 0 - 9. <br>
Using just two PCs, we get the following clusters. 

In [ ]:
plt.scatter(projected[:, 0], projected[:, 1],
            c=digits.target, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('Spectral', 10))

plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

Each image in the training set is defined by a collection of 64 pixel values, which we will call the vector  𝑥 :

𝑥=[𝑥1,𝑥2,𝑥3⋯𝑥64] 

One way we can think about this is in terms of a pixel basis. That is, to construct the image, we multiply each element of the vector by the pixel it describes, and then add the results together to build the image:

image(𝑥)=𝑥1⋅(pixel 1)+𝑥2⋅(pixel 2)+𝑥3⋅(pixel 3)⋯𝑥64⋅(pixel 64)

In [ ]:
from IPython.display import Image
Image("pixels.png" , width=640)

But the pixel-wise representation is not the only choice of basis. We can also use other basis functions, which each contain some pre-defined contribution from each pixel, and write something like

$$
image(x) = {\rm mean} + x_1 \cdot{\rm (basis~1)} + x_2 \cdot{\rm (basis~2)} + x_3 \cdot{\rm (basis~3)} \cdots
$$

PCA can be thought of as a process of choosing optimal basis functions, such that adding together just the first few of them is enough to suitably reconstruct the bulk of the elements in the dataset.
The principal components, which act as the low-dimensional representation of our data, are simply the coefficients that multiply each of the elements in this series.
This figure shows a similar depiction of reconstructing this digit using the mean plus the first eight PCA basis functions:

In [ ]:
Image("pca.png" , width=640)

In [ ]:
pca = PCA().fit(digits.data)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

**Training the model on the principal components**

In [ ]:
model = DecisionTreeClassifier()
#List of (name, transform) tuples (implementing fit/transform) that are chained, 
#in the order in which they are chained, with the last object an estimator.
pipeline = Pipeline(steps=[('p',pca),('m',model)])
pipeline.fit(projected, y)
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, projected, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# **Assignment**
The first model, model_orig was trained using all 64 pixels. It had an 85% accuracy score. <br>

The second model, model, was trained using the first two principal components. It had an accuracy of 58%

Your assignment: Modify the code for PCA from 2 inputs to more inputs. Try to increase the accuracy of the model using different PCs. 